In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import tensorflow_hub as hub
import random
from tensorflow.keras.layers.experimental import preprocessing


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/paddy-disease-classification/train.csv")
samle_sub = pd.read_csv("/kaggle/input/paddy-disease-classification/sample_submission.csv")
display(train.head(), train.info())

In [ ]:
# Visualize label column
plt.figure(figsize=(15,10))
ax = sns.countplot(train.label)
for p in ax.patches:
    percentage = f"{(100 * p.get_height()/len(train)):.2f}" + "%"
    ax.annotate(percentage, (p.get_x()+0.3, p.get_height()), ha = "center", va = "top", c = "white", fontsize = 14 )
    ax.annotate(p.get_height(), (p.get_x()+0.3, p.get_height()-100), ha = "center", va = "top", c = "white", fontsize = 14)
plt.title("percentages & counts of classes")
plt.xticks(rotation = 45);

In [ ]:
# Get train and test dir
train_dir = pathlib.Path("/kaggle/input/paddy-disease-classification/train_images")
test_dir = pathlib.Path("/kaggle/input/paddy-disease-classification/test_images")
print(f"There are {len(list(train_dir.glob('*/*.jpg')))} images belongs to {len(list(train_dir.glob('*/')))} classes in Train directory ")

In [ ]:
# Create a function for generating random image

def random_image(class_name):
    label = list(train_dir.glob(f'{class_name}/*'))
    random_choice = random.choice(label)
    return PIL.Image.open(str(random_choice))

In [ ]:
# Get datasets for Transfer Learning
BATCH_SIZE = 16
IMAGE_SIZE = (224,224)

train_dataset = tf.keras.utils.image_dataset_from_directory("/kaggle/input/paddy-disease-classification/train_images/",
                                                            labels='inferred',
                                                            label_mode = "categorical",
                                                            #class_names = train.label.unique(),
                                                            color_mode = "rgb",
                                                            shuffle = True,
                                                            seed = 42,
                                                            validation_split = 0.1,
                                                            subset = "training",
                                                            batch_size = BATCH_SIZE,
                                                            image_size = IMAGE_SIZE
                                                            ).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.keras.utils.image_dataset_from_directory("/kaggle/input/paddy-disease-classification/train_images/",
                                                            labels='inferred',
                                                            label_mode = 'categorical',
                                                            #class_names = train.label.unique(),
                                                            color_mode = "rgb",
                                                            shuffle = True,
                                                            seed = 42,
                                                            validation_split = 0.1,
                                                            subset = "validation",
                                                            batch_size = BATCH_SIZE,
                                                            image_size = IMAGE_SIZE
                                                            ).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.3),
    preprocessing.RandomZoom(0.2)
])

In [ ]:
# Get model for Transfer Learning 
effnet_model = tf.keras.applications.EfficientNetB4(include_top = False,
                                                    input_shape = IMAGE_SIZE+(3,),
                                                    #pooling = "avg",
                                                   )

In [ ]:
# Functional API
effnet_model.trainable = False


inputs = layers.Input(shape = IMAGE_SIZE+(3,))
data_aug = data_augmentation(inputs)
effnet_layer = effnet_model(data_aug)
x = layers.Dense(512, activation = "relu")(effnet_layer)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation = "relu")(x)
x = layers.Dropout(0.2)(x)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(10, activation = "softmax")(x)
model = tf.keras.Model(inputs, outputs)
# compile 
model.compile(loss = "categorical_crossentropy",
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ["accuracy"])

In [ ]:
model.summary()

In [ ]:
# Plot model
plot_model(effnet_model)

In [ ]:
# Check layers are trainable 
for layer in effnet_model.layers:
    layer.trainable = False

In [ ]:
# Create history 
initial_epochs = 20
history_model = model.fit(train_dataset,
                         epochs = initial_epochs,
                         validation_data = val_dataset,
                         callbacks = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", patience = 3, restore_best_weights = True))

In [ ]:
# Plot Accuracy and Loss Curves
## Loss
plt.figure(figsize = (10,7))
plt.plot(history_model.history["loss"], label = "Training Loss")
plt.plot(history_model.history["val_loss"], label = "Validation Loss")
plt.title("Train, Val Loss")
plt.ylabel("loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.legend();

## Accuracy
plt.figure(figsize = (10,7))
plt.plot(history_model.history["accuracy"], label = "Training Accuracy")
plt.plot(history_model.history["val_accuracy"], label = "Validation Accuracy")
plt.title("Train, Val Accuracy")
plt.ylabel("loss")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.legend();

## Fine Tunning

In [ ]:
# Turn on trainable layers of last 10 layer
model.trainable = True
for layer in model.layers[:-10]:
    layer.trainable = False

# re-compile is needed
model.compile(loss = "categorical_crossentropy",
                    optimizer = tf.keras.optimizers.Adam(),
                    metrics = ["accuracy"])

In [ ]:
plot_model(effnet_model)

In [ ]:
# Fine tune for another 10 epochs 
finetune_epochs = initial_epochs + 30

# Create fine tune history
history_finetune = model.fit(train_dataset,
                                   epochs = finetune_epochs,
                                   initial_epoch = history_model.epoch[-1],
                                   validation_data = val_dataset,
                                   callbacks = [tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", patience = 3, restore_best_weights = True),
                                                tf.keras.callbacks.ModelCheckpoint("./checkpoint", monitor = "val_accuracy",save_best_only=False,save_weights_only=False)])

In [ ]:
import h5py

model_json = model.to_json()
with open("model_fine.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_fine.h5")
print("Saved model to disk")